
This page will take you through pulling a SQL Server image down from the Microsoft Container Registry and spinning up a container.
We used to be able to search the Docker Hub for SQL Server container images: -



In [0]:
    docker search mssql



However, Microsoft have moved their container images to the Microsoft Container Registry (MCR) so now we have to run the following: -



In [0]:
    $repo1 = invoke-webrequest https://mcr.microsoft.com/v2/mssql/server/tags/list
    $repo2 = invoke-webrequest https://mcr.microsoft.com/v2/mssql/rhel/server/tags/list
    $tags = $repo1.content + $repo2.content
    $tags



This will show us all the Ubuntu and Red Hat SQL Server container images that are available to us.
So now we can pull down an image with: -



In [0]:
    docker image pull mcr.microsoft.com/mssql/server:2019-CU5-ubuntu-18.04



![](../images/2.RunningYourFirstContainer/RunningAContainer/1.docker_image_pull.png)
Here we're pulling down SQL Server 2019 CU5 Ubuntu 18.04 image. I know we could just put the image into our docker container run statement but I like to pull my images down first.
Once the pull is complete we can verify that the image is on our local machine with: -



In [0]:
    docker image ls



![](../images/2.RunningYourFirstContainer/RunningAContainer/2.docker_image_ls.png)
So now we can run a container from that image: -



In [0]:
    docker container run -d `
    -p 15789:1433 `
    --env ACCEPT_EULA=Y `
    --env MSSQL_SA_PASSWORD=Testing1122 `
    --name sqlcontainer1 `
    mcr.microsoft.com/mssql/server:2019-CU5-ubuntu-18.04



![](../images/2.RunningYourFirstContainer/RunningAContainer/3.docker_container_run.png)
Here we are spinning up a container called sqlcontainer1, accepting the EULA, setting the SA pasword to Testing1122, and mapping port 15789 on our host to port 1433 within the container.
But before we connect, we can check that the container is up and running with: -



In [0]:
    docker container ls -a



![](../images/2.RunningYourFirstContainer/RunningAContainer/4.docker_container_ls.png)
The -a flag shows all containers and the [container ls]( https://docs.docker.com/engine/reference/commandline/container_ls/) command by default will only show running containers.
If we want to clean up the output of that a little bit we could run: -



In [0]:
    docker container ls -a --format "table {{.Names }}\t{{ .Image }}\t{{ .Status }}\t{{.Ports}}"



![](../images/2.RunningYourFirstContainer/RunningAContainer/5.docker_container_ls2.png)
I'll go through this in more detail in a later wiki page but for now, let's move on and check the SQL logs: -



In [0]:
    docker container logs sqlcontainer1



![](../images/2.RunningYourFirstContainer/RunningAContainer/6.docker_container_logs.png)
Cool! A nice and easy way to view the SQL Server error log!
Let's connect to SQL. We could connect in SSMS with localhost,15789 or with the [mssql-cli](https://docs.microsoft.com/en-us/sql/tools/mssql-cli): -



In [0]:
    mssql-cli -S 'localhost,15789' -U sa -P Testing1122 -Q "SELECT @@VERSION AS [Version]"



![](../images/2.RunningYourFirstContainer/RunningAContainer/7.mssql_cli_version.png)
If we wanted to jump into the container itself, we can run: -



In [0]:
    docker container exec -it sqlcontainer1 bash



And then run the following to have a look at the system databases: -



In [0]:
    cd /var/opt/mssql/data
    ls -al
    exit



![](../images/2.RunningYourFirstContainer/RunningAContainer/8.docker_exec.png)
But we don't have to jump into the container at all to run those commands, we could instead run: -



In [0]:
    docker exec sqlcontainer1 ls -al /var/opt/mssql/data
